In [ ]:
import undetected_chromedriver.v2 as uc
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from importlib.machinery import SourceFileLoader
import time
from selenium import webdriver
import json

In [ ]:
driver = uc.Chrome()
base_url = 'https://www.johnlewis.com/'
driver.get(base_url)

In [ ]:
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, 'c-button-yMKB7')))
accept_cookies = driver.find_elements(By.CLASS_NAME, 'c-button-yMKB7')
if len(accept_cookies) > 0:
    accept_cookies[0].click()

In [ ]:
def getFilename(conf):
    file_name = page_config['country'] + '_' + page_config['retailer'] + '_' + page_config['department'] + '_' + page_config['category'] + '_' + page_config['page']
    for k, v in json.load(open('config/encoding.json', 'r')).items():
        file_name = file_name.replace(k, v)
        file_name = file_name.replace('\\', '')
    return file_name


conf = {}
conf['country'] = 'United Kingdom'
conf['retailer'] = 'John Lewis'
conf['parsing_config'] = './retailer_config/johnlewis_parsing.json'
conf['gcloud_path'] = 'crawl_data/United Kingdom/John_Lewis/'
page_configs = []

In [ ]:
departments = driver.find_elements(By.CLASS_NAME, 'DesktopMenuItem-desktopMenuItem--75bbf')
departments_texts = []

for did, department in enumerate(departments[3:5]):
    action = ActionChains(driver)
    action.move_to_element(department).perform()
    time.sleep(1)
    department_name = department.find_element(By.TAG_NAME, 'a').text
    department_url = department.find_element(By.TAG_NAME, 'a').get_attribute('href')
    
    page_config = conf.copy()            
    page_config['department'] = department_name
    page_config['category'] = department_name
    page_config['page'] = department_name
    page_config['page_url'] = department_url
    page_config['file_name'] = getFilename(page_config)
    page_configs.append(page_config)
    
    print(department_name + ' | ' + department_name + ' | ' + department_name)
    print(department_url)
    
    splits = department.find_elements(By.XPATH, './div/div/ul')
    categories = []
    for split in splits:
        categories.extend(split.find_elements(By.XPATH, './li'))
        
    for category in categories:
        category_name = category.find_element(By.TAG_NAME, 'h4').text
        
        if(category.find_elements(By.XPATH, './a')):
            print(department_name + ' | ' + category_name + ' | ' + category_name)
            print(category.find_element(By.XPATH, './a').get_attribute('href'))
        
        pages = category.find_elements(By.XPATH, './ul/li/a')
        for page in pages:
            page_name = page.text
            page_url = page.get_attribute('href')
            
            page_config = conf.copy()            
            page_config['department'] = department_name
            page_config['category'] = category_name
            page_config['page'] = page_name
            page_config['page_url'] = page_url
            page_config['file_name'] = getFilename(page_config)
            page_configs.append(page_config)  
            print(department_name + ' | ' + category_name + ' | ' + page_name)
            print(page_url)
            

In [ ]:
with open("johnlewiscom_urls.json", "w") as outfile:
        json.dump(page_configs, outfile)

In [ ]:
len(page_configs)
for conf in page_configs:
    conf['country'] = 'United Kingdom'
    conf['retailer'] = 'John Lewis'
    conf['parsing_config'] = './retailer_config/johnlewis_parsing.json'
    conf['gcloud_path'] = 'crawl_data/United Kingdom/John_Lewis/'